# Experiments with LangChain

In [2]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

res = chain.run("weed")
res



> Entering new LLMChain chain...
Prompt after formatting:
What is a good name for a company that makes weed?

> Finished chain.


'\n\nGreen Harvest Cannabis Co.'

In [13]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory(),
    verbose=True
)
conversation.run("Answer briefly. What are the first 3 colors of a rainbow?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Answer briefly. What are the first 3 colors of a rainbow?
AI:

> Finished chain.


'The first three colors of a rainbow are red, orange, and yellow.'

In [14]:
# chain could persist memory across calls
conversation.run("And the next 4?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Answer briefly. What are the first 3 colors of a rainbow?
AI: The first three colors of a rainbow are red, orange, and yellow.
Human: And the next 4?
AI:

> Finished chain.


'The next four colors of a rainbow are green, blue, indigo, and violet.'

In [15]:
from langchain.chains import SimpleSequentialChain

second_prompt = PromptTemplate(
    input_variables=['company'],
    template="Write a catchphrase for the following company {company}",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)
overall_chain.run("colorful socks")



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
What is a good name for a company that makes colorful socks?

> Finished chain.


Bright Footsocks.


"Step into the Bright Side with Bright Footsocks!"

> Finished chain.


'\n\n"Step into the Bright Side with Bright Footsocks!"'

In [6]:
with open("data/joe_biden_cancer_moonshot.txt") as f:
    joe_biden = f.read()

In [11]:
# split the documents
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document

# what are different token splitter types?
text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=30)
texts = Document(page_content=joe_biden)
texts = text_splitter.split_documents([texts])
texts

[Document(page_content='Remarks by Vice President Joe Biden on The Cancer Moonshot\nWorld Economic Forum\nDavos, Switzerland\n\nTHE VICE PRESIDENT:  Madam Ambassador, thanks for putting together a crowd for us.  (Laughter.) \n\nFolks, I’m delighted to be here and happy to be back with so many distinguished people.  And I mean that sincerely.  Many of you know as much or more about the subjects I’m going to speak to than I do, including -- is Elizabeth Blackburn here?  Elizabeth, a Nobel Laureate, who found the BRCA gene.  As that old saying goes, she’s forgotten more about this than I’m going to know.\n\nBut, folks, I’m happy to be back here at the World Economic Forum at Davos to talk about the fight against cancer.  And I’m accompanied by Greg Simon, who is Executive Director of administration’s Cancer Moonshot and who will lead my Biden Cancer Initiative that I’ll launch after we leave office, which is in about 48 hours.  (Laughter.)  I hope I have a ride home.  (Laughter.)  I’ll ta

In [12]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import AnalyzeDocumentChain

llm = OpenAI(temperature=0)
# for each docucment, summarize first - then combine with another summary over all summaries
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")
summary_chain({"input_documents": texts})["output_text"]

' Vice President Joe Biden has been traveling the world to discuss the fight against cancer and has established a White House Cancer Moonshot Task Force to break down silos in the federal government. He has also established a Biden Cancer Initiative to continue the work of ending cancer as we know it. He encourages collaboration between cancer centers, drug companies, the insurance industry, and government in order to end cancer as we know it. He is optimistic that with the commitment and brilliance of researchers and scientists, patients will one day get the right therapy the first time for their cancer.'

In [20]:
# alternatively, AnalyzeDocumentChain has text splitting logics built in
summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)
summarize_document_chain.run(joe_biden)

" Vice President Joe Biden spoke at the World Economic Forum in Davos, Switzerland about the Cancer Moonshot initiative, which aims to double the rate of progress in the fight against cancer. He discussed the progress made since President Nixon declared a War on Cancer in 1971, including the development of immunotherapy, robotics, and liquid biopsies. He also discussed the collaborations and initiatives launched since the Cancer Moonshot Summit, such as IBM's supercomputer Watson, the National Cancer Institute Genomic Data Commons, and the Centers for Disease Control and Prevention's cancer vaccines. He encouraged the audience to double down on their efforts to find a solution and expressed gratitude for their willingness to listen."

In [25]:
from langchain.chains.question_answering import load_qa_chain

# qa chain map reduce
# map: for each context, extract out relevant part based on question; leave out rest
# reduce: for all relevent context, formulate an answer
qa_chain = load_qa_chain(llm, chain_type="map_reduce", verbose=True)
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)
qa_document_chain.run(input_document=joe_biden, question="what did the president say about justice breyer?")



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
Remarks by Vice President Joe Biden on The Cancer Moonshot
World Economic Forum
Davos, Switzerland

THE VICE PRESIDENT:  Madam Ambassador, thanks for putting together a crowd for us.  (Laughter.) 

Folks, I’m delighted to be here and happy to be back with so many distinguished people.  And I mean that sincerely.  Many of you know as much or more about the subjects I’m going to speak to than I do, including -- is Elizabeth Blackburn here?  Elizabeth, a Nobel Laureate, who found the BRCA gene.  As that old saying goes, she’s forgotten more about this than I’m going to know.

But, folks, I’m happy to be back here at the World Economic Forum at Davos to talk about the fight against cancer.  And I’m accompanied by Greg Simon, who is Executive Director of administration’s Cance

' The president did not mention Justice Breyer.'